### Install Dependencies

In [64]:
#%pip install pandas
#%pip install matplotlib
#%pip install python-dotenv
#%pip install coinbase-advanced-py

### Import dependencies

In [65]:
import os
import pandas as pd  
import matplotlib.pyplot as plt
from coinbase.rest import RESTClient
from dotenv import load_dotenv
import json
import time
import matplotlib.pyplot as plt
from datetime import datetime, time

client = RESTClient() # Uses environment variables for API key and secretimport os

In [66]:
load_dotenv()

api_secret = os.getenv('CB_PRIVATE_KEY')
api_key = os.getenv('CB_KEY_NAME')

client = RESTClient(api_key=api_key, api_secret=api_secret) # Uses environment variables for API key and secretimport os


In [67]:
# Provided datetime string
date_string = "2025-03-14 23:04:16.929545+00:00"

# Convert to a datetime object
dt = datetime.fromisoformat(date_string)

# Convert to Unix timestamp in seconds
current_time = int(dt.timestamp())
print(current_time)

1741993456


In [68]:
total_trades = []
start_time = 1739660656
end_time = 1741993456
limit = end_time - start_time
interval = 15
count = 0
success = 0

while start_time < end_time:
    count += 1
    trades = client.get_market_trades('ETH-USD', 1, start_time, start_time + 1)
    try:
        total_trades.append(trades['trades'][0])
        start_time = start_time + interval
        success += 1
    except IndexError as e:
        start_time = start_time + 1
    
    if(count % 1000 == 0):
        print(f'Completed {count+1}/{limit} queries. Successful queries: {success}')

Completed 1001/2332800 queries. Successful queries: 829
Completed 2001/2332800 queries. Successful queries: 1555
Completed 3001/2332800 queries. Successful queries: 2249
Completed 4001/2332800 queries. Successful queries: 2814
Completed 5001/2332800 queries. Successful queries: 3420
Completed 6001/2332800 queries. Successful queries: 4135
Completed 7001/2332800 queries. Successful queries: 4885
Completed 8001/2332800 queries. Successful queries: 5664
Completed 9001/2332800 queries. Successful queries: 6422
Completed 10001/2332800 queries. Successful queries: 7120
Completed 11001/2332800 queries. Successful queries: 7782
Completed 12001/2332800 queries. Successful queries: 8503
Completed 13001/2332800 queries. Successful queries: 9290
Completed 14001/2332800 queries. Successful queries: 10194
Completed 15001/2332800 queries. Successful queries: 11029
Completed 16001/2332800 queries. Successful queries: 11838
Completed 17001/2332800 queries. Successful queries: 12650
Completed 18001/2332

KeyboardInterrupt: 

In [2]:
len(total_trades)

NameError: name 'total_trades' is not defined

In [74]:
str_trades = str(total_trades)
# Replace single quotes with double quotes to make it JSON-compatible
data_string_fixed = str_trades.replace("'", '"')

# Convert the string to a proper Python list of dictionaries
data_list = json.loads(data_string_fixed)

In [59]:
df_trades = pd.DataFrame(data_list)
df_trades

trade_id product_id     price  \
0                                          610378394    ETH-USD   2693.31   
1                                          610378498    ETH-USD   2693.07   
2                                          610378533    ETH-USD   2693.89   
3                                          610378584    ETH-USD   2693.66   
4                                          610378657    ETH-USD   2693.98   
...                                              ...        ...       ...   
5390                                       610613414    ETH-USD   2682.26   
5391                                       610613454    ETH-USD   2682.16   
5392                                       400308574    ETH-USD      2681   
5393                                       400308589    ETH-USD    2680.5   
5394  2025-02-16T22:10:48.827Z-ETH/USD-2680.751-0.04    ETH-USD  2680.751   

            size                         time                side exchange  \
0     0.00266163  2025-02-15T23:04:16.929545Z                SELL            
1     0.00329429  2025-02-15T23:04:31.279857Z                SELL            
2     0.00324313  2025-02-15T23:04:46.953075Z                SELL            
3     0.00229312  2025-02-15T23:05:01.536506Z                SELL            
4     0.00053698  2025-02-15T23:05:16.783522Z                SELL            
...          ...                          ...                 ...      ...   
5390  0.00099657  2025-02-16T22:09:48.990624Z                 BUY            
5391  0.11077308  2025-02-16T22:10:03.350756Z                SELL            
5392  0.00520294     2025-02-16T22:10:18.750Z                 BUY            
5393  0.00543836     2025-02-16T22:10:33.919Z                 BUY            
5394        0.04     2025-02-16T22:10:48.827Z  UNKNOWN_ORDER_SIDE            

     bid ask  
0             
1             
2             
3             
4             
...   ..  ..  
5390          
5391          
5392          
5393          
5394          

[5395 rows x 9 columns]

In [71]:
# Convert 'time' to datetime, coercing errors
df_trades['time'] = pd.to_datetime(df_trades['time'], errors='coerce')

# Drop rows where 'time' is NaT (invalid date)
df_trades = df_trades.dropna(subset=['time'])

# Convert 'price' column to numeric
df_trades['price'] = pd.to_numeric(df_trades['price'])

# Plot the data
plt.figure(figsize=(10, 6))
plt.plot(df_trades['time'], df_trades['price'], linestyle='-', label='Price')

# Add labels and title
plt.xlabel('Time')
plt.ylabel('Price (USD)')
plt.title('ETH-USD Price Over Time')
plt.grid(True)
plt.legend()
plt.tight_layout()

# Show the plot
plt.show()

/var/folders/1f/5lhk7sb97rz53v7gk1vkcjkr0000gn/T/ipykernel_54541/3006657511.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trades['time'] = pd.to_datetime(df_trades['time'], errors='coerce')


In [73]:
df_trades.to_csv('recent_eth_prices.csv')